## Avoid too much similarity among the images: skip images while copying 

In [6]:
import glob
import os
image_dir = '/home/cloudlet/datasets/argoverse-tracking/sample/c6911883-1843-3727-8eaa-41dc8cda8993/'
output_dir_db = '/home/cloudlet/datasets/argoverse-tracking/sample/test_1017_2/db/'
if not os.path.exists(output_dir_db):
    os.makedirs(output_dir_db)
output_dir_query = '/home/cloudlet/datasets/argoverse-tracking/sample/test_1017_2/query/'
if not os.path.exists(output_dir_query):
    os.makedirs(output_dir_query)
    
task_list_db = ['ring_front_right', 'ring_front_left', 'ring_front_center', 'ring_rear_right', 'ring_rear_left']
skip = 3
for task in task_list_db:
    counter = 0
    for filename in glob.glob(image_dir + task + '/*.jpg'):
        if counter % skip == 0:
            cmd = 'cp '+ filename + ' ' + output_dir_db
            os.system(cmd)
        counter += 1
    print('For task ' + task + ' copied ',  counter//skip + 1, 'images')

task_list_query = ['ring_side_right', 'ring_side_left', 'stereo_front_right', 'stereo_front_left']
skip = 2
for task in task_list_query:
    counter = 0
    for filename in glob.glob(image_dir + task + '/*.jpg'):
        if counter % skip == 0:
            cmd = 'cp '+ filename + ' ' + output_dir_query
            os.system(cmd)
        counter += 1
    print('For task ' + task + ' copied ',  counter//skip + 1, 'images')

output_dir_db_query = '/home/cloudlet/datasets/argoverse-tracking/sample/test_1017_2/db_query/'
if not os.path.exists(output_dir_db_query):
    os.makedirs(output_dir_db_query)
cmd = 'cp '+ output_dir_db + '/*.jpg ' + output_dir_db_query
os.system(cmd)
cmd = 'cp '+ output_dir_query + '/*.jpg ' + output_dir_db_query
os.system(cmd)

For task ring_front_right copied  157 images
For task ring_front_left copied  157 images
For task ring_front_center copied  157 images
For task ring_rear_right copied  157 images
For task ring_rear_left copied  157 images
For task ring_side_right copied  235 images
For task ring_side_left copied  235 images
For task stereo_front_right copied  40 images
For task stereo_front_left copied  40 images


0

In [1]:
import cv2
import numpy as np
import tensorflow as tf

import netvlad_tf.net_from_mat as nfm
import netvlad_tf.nets as nets

tf.reset_default_graph()

image_batch = tf.placeholder(
        dtype=tf.float32, shape=[None, None, None, 3])

net_out = nets.vgg16NetvladPca(image_batch)
saver = tf.train.Saver()

sess = tf.Session()
saver.restore(sess, nets.defaultCheckpoint())

inim = cv2.imread(nfm.exampleImgPath())
inim = cv2.cvtColor(inim, cv2.COLOR_BGR2RGB)
print(inim.shape)
batch = np.expand_dims(inim, axis=0)
print(batch.shape)
result = sess.run(net_out, feed_dict={image_batch: batch})
print(result.shape)

/home/cloudlet/anaconda3/envs/hfnet/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/cloudlet/anaconda3/envs/hfnet/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/cloudlet/anaconda3/envs/hfnet/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/cloudlet/anaconda3/envs/hfnet/lib/pyt

Instructions for updating:
dim is deprecated, use axis instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
INFO:tensorflow:Restoring parameters from /home/cloudlet/work/netvlad_tf_open/checkpoints/vd16_pitts30k_conv5_3_vlad_preL2_intra_white
(201, 300, 3)
(1, 201, 300, 3)
(1, 4096)


In [2]:
import glob
from tqdm import tqdm
result_list = []
filename_list = []
dataset_path = '/home/cloudlet/datasets/argoverse-tracking/sample/test_1017_2/'
for filename in tqdm(glob.glob(dataset_path + 'db/*.jpg')):
    filename_list.append(filename.split('/')[-1])
    inim = cv2.imread(filename)
    inim = cv2.cvtColor(inim, cv2.COLOR_BGR2RGB)
#     print(inim.shape)
    batch = np.expand_dims(inim, axis=0)
#     print(batch.shape)
    result = sess.run(net_out, feed_dict={image_batch: batch})
    result_list.append(result)
print(filename_list[5])
print(len(filename_list))

100%|██████████| 785/785 [01:47<00:00,  7.34it/s]

ring_rear_right_315978414491055520.jpg
785


In [9]:
result_list_query = []
filename_list_query = []
for filename in tqdm(glob.glob(dataset_path + 'query/*.jpg')):
    filename_list_query.append(filename.split('/')[-1])
    inim = cv2.imread(filename)
    inim = cv2.cvtColor(inim, cv2.COLOR_BGR2RGB)
#     print(inim.shape)
    batch = np.expand_dims(inim, axis=0)
#     print(batch.shape)
    result = sess.run(net_out, feed_dict={image_batch: batch})
    result_list_query.append(result)
print(filename_list_query[5])
print(len(filename_list_query))

100%|██████████| 548/548 [01:33<00:00,  5.86it/s]

stereo_front_left_315978407897654688.jpg
548


In [12]:
# this is for the matching test 10.27
# Query image
base_dir = '/home/cloudlet/work/Hierarchical-Localization/test_query/'
result_list_query = []
filename_list_query = []
for filename in tqdm(glob.glob(base_dir + 'query_image/*.jpg')):
    filename_list_query.append(filename.split('/')[-1])
    inim = cv2.imread(filename)
    inim = cv2.cvtColor(inim, cv2.COLOR_BGR2RGB)
#     print(inim.shape)
    batch = np.expand_dims(inim, axis=0)
#     print(batch.shape)
    result = sess.run(net_out, feed_dict={image_batch: batch})
    result_list_query.append(result)
print(filename_list_query)
print(len(filename_list_query))

100%|██████████| 1/1 [00:00<00:00,  6.26it/s]

['query.jpg']
1


In [13]:
def compute_distance(desc1, desc2):
    # For normalized descriptors, computing the distance is a simple matrix multiplication.
    return 2 * (1 - desc1 @ desc2.T)

In [14]:
db = np.concatenate(result_list, axis=0)
print(db.shape)

(785, 4096)


## Generate pair files

In [15]:
# this is for the matching test 10.27
pair_path = base_dir + 'pairs/'
top_k = 20
import os
if not os.path.exists(pair_path):
    os.makedirs(pair_path)
with open(pair_path + 'pairs'+str(top_k)+'_argo_query.txt', 'w') as file:
    for i1, query in enumerate(result_list_query):
        query = query.flatten()
        dist = compute_distance(query, db)
        index_min_k = np.argsort(dist)[:top_k]

        for i2, db_file_index in enumerate(index_min_k):
            if i2 == len(index_min_k)-1 and i1 == len(result_list_query)-1:
                line = filename_list_query[i1] + ' ' + filename_list[db_file_index]
            else:
                line = filename_list_query[i1] + ' ' + filename_list[db_file_index] + '\n'
            file.write(line)

In [15]:
pair_path = '/home/cloudlet/work/Hierarchical-Localization/pairs/argo_1017_2/'
top_k = 60
import os
if not os.path.exists(pair_path):
    os.makedirs(pair_path)
with open(pair_path + 'pairs'+str(top_k)+'_argo_query.txt', 'w') as file:
    for i1, query in enumerate(result_list_query):
        query = query.flatten()
        dist = compute_distance(query, db)
        index_min_k = np.argsort(dist)[:top_k]

        for i2, db_file_index in enumerate(index_min_k):
            if i2 == len(index_min_k)-1 and i1 == len(result_list_query)-1:
                line = filename_list_query[i1] + ' ' + filename_list[db_file_index]
            else:
                line = filename_list_query[i1] + ' ' + filename_list[db_file_index] + '\n'
            file.write(line)

In [16]:
top_k = 60
with open(pair_path + 'pairs'+str(top_k)+'_argo_db.txt', 'w') as file:
    for i1, db_image in enumerate(result_list):
        db_image = db_image.flatten()
        dist = compute_distance(db_image, db)
        index_min_k = np.argsort(dist)[1:top_k+1]

        for i2, db_file_index in enumerate(index_min_k):
            if i2 == len(index_min_k)-1 and i1 == len(result_list)-1:
                line = filename_list[i1] + ' ' + filename_list[db_file_index]
            else:
                line = filename_list[i1] + ' ' + filename_list[db_file_index] + '\n'
            file.write(line)

## Generate queries_with_intrinsics.txt for localization.py

In [14]:
import glob
from tqdm import tqdm
dataset_path = '/home/cloudlet/datasets/argoverse-tracking/sample/test_1017_2/'
with open(dataset_path + 'queries_with_intrinsics_1017_2.txt', 'w') as file:
    image_glob_list = glob.glob(dataset_path + 'query/*.jpg')
    for i, filename in enumerate(image_glob_list):
        name = filename.split('/')[-1]
        if 'ring_rear' in name:
            params = '1398.898 956.8 580.9 -0.1681239'
# "focal_length_x_px_": 1398.8981635526468, "focal_length_y_px_": 1398.8981635526468, 
# "focal_center_x_px_": 956.7924764366256, "focal_center_y_px_": 580.9004713513332, 
# "skew_": 0.0, "distortion_coefficients_": [-0.16812389660709987, 0.1156063782256891, -0.021955074613449775]
        elif 'ring_side' in name:
            params = '1406.669 969.1 622.6 -0.1700210'
# "focal_length_x_px_": 1406.668708764027, "focal_length_y_px_": 1406.668708764027, 
# "focal_center_x_px_": 969.1274587919514, "focal_center_y_px_": 622.6176623508292, "skew_": 0.0, 
# "distortion_coefficients_": [-0.170020986074912, 0.11822570242692526, -0.024028975508574642]
        elif 'stereo_front_left' in name:
            params = '3666.738 1230.5 1059.5 -0.1073885'
# "focal_length_x_px_": 3666.737987697073, "focal_length_y_px_": 3666.737987697073, 
# "focal_center_x_px_": 1230.5215533709008, "focal_center_y_px_": 1059.5814244164, "skew_": 0.0, 
# "distortion_coefficients_": [-0.10738858957839269, 0.19257998343246094, -0.004826963862134801]
        elif 'stereo_front_right' in name:
            params = '3663.383 1245.1 1073.9 -0.1057950'
# "focal_length_x_px_": 3663.3831779453944, "focal_length_y_px_": 3663.3831779453944, 
# "focal_center_x_px_": 1245.152498763069, "focal_center_y_px_": 1073.9194352453455, "skew_": 0.0, 
# "distortion_coefficients_": [-0.10579500162786715, 0.15297214630504186, 0.1282398068572198]
        else:
            print('error when reading params')
            
        end_str = '' if i == len(image_glob_list)-1 else '\n'       
        if 'ring_' in name:
            line = name + ' SIMPLE_RADIAL 1920 1200 ' + params + end_str
        elif 'stereo_' in name:
            line = name + ' SIMPLE_RADIAL 2464 2056 ' + params + end_str
        else:
            print('error when reading cam type')
        file.write(line)


## Generate the Ground Truth Camera Position for db image
So that we could use COLMAP model_aligner to rescale the 3d model

In [1]:
import glob
import json
from tqdm import tqdm
from scipy.spatial.transform import Rotation
import numpy as np

def tf_matrix(qvec, tvec):
    # suppose argoverse is [qw,qx,qy,qz], while scipy is [qx,qy,qz,qw]
    R = Rotation.from_quat([qvec[1], qvec[2], qvec[3], qvec[0]]).as_matrix()
    t = np.array(tvec).reshape(3,1)
    T = np.concatenate((R, t), axis=1)
    T = np.concatenate((T, np.array([[0,0,0,1]])), axis=0)
#     print(T)
    return T
    
# tf_matrix([-0.3662878606998999, 0.006933435549559967, 0.006966299178769239, 0.930449676904196],\
#           [2601.869677585785, 1211.1693848655868, 15.694929787299001])

In [2]:
import numpy as np
calib_file_dir = '/home/cloudlet/datasets/argoverse-tracking/sample/c6911883-1843-3727-8eaa-41dc8cda8993/vehicle_calibration_info.json'
T_vc_front_left = {}
T_vc_front_right = {}
T_vc_front_center = {}
T_vc_rear_left = {}
T_vc_rear_right = {}
with open(calib_file_dir) as json_file: 
    calib = json.load(json_file) 
#     print(calib['camera_data_'])
    for item in calib['camera_data_']:
        if 'ring_front_left' in item['key']:
            pose =  item['value']['vehicle_SE3_camera_']
            T_vc_front_left = {'rotation': pose['rotation']['coefficients'],\
                               'translation': pose['translation']}

        if 'ring_front_right' in item['key']:
            pose =  item['value']['vehicle_SE3_camera_']
            T_vc_front_right = {'rotation': pose['rotation']['coefficients'],\
                               'translation': pose['translation']}
        if 'ring_front_center' in item['key']:
            pose =  item['value']['vehicle_SE3_camera_']
            T_vc_front_center = {'rotation': pose['rotation']['coefficients'],\
                               'translation': pose['translation']}

        if 'ring_rear_left' in item['key']:
            pose =  item['value']['vehicle_SE3_camera_']
            T_vc_rear_left = {'rotation': pose['rotation']['coefficients'],\
                               'translation': pose['translation']}
            
        if 'ring_rear_right' in item['key']:
            pose =  item['value']['vehicle_SE3_camera_']
            T_vc_rear_right = {'rotation': pose['rotation']['coefficients'],\
                               'translation': pose['translation']}

In [3]:
dataset_path = '/home/cloudlet/datasets/argoverse-tracking/sample/test_1017_2/'
image_dir = dataset_path + 'db/'
pose_dir = '/home/cloudlet/datasets/argoverse-tracking/sample/c6911883-1843-3727-8eaa-41dc8cda8993/poses/'
count = 0

with open(dataset_path + 'db_location_1017_2.txt', 'w') as location_file:
    for filename in tqdm(glob.glob(image_dir + '*.jpg')):
        img_name = filename.split('/')[-1]
        ts = img_name.split('_')[-1]
        if ts.endswith('.jpg'):
            ts = ts[:-4]
        else:
            print('wrong image name format')

        with open(pose_dir + 'city_SE3_egovehicle_' + ts + '.json') as json_file: 
            pose_wv = json.load(json_file) # vehicle to world
            qvec = pose_wv['rotation']
            tvec = pose_wv['translation']
            T_wv = tf_matrix(qvec,tvec)
            if 'front_left' in img_name:
                pose_vc = T_vc_front_left
            elif 'front_right' in img_name:
                pose_vc = T_vc_front_right
            elif 'front_center' in img_name:
                pose_vc = T_vc_front_center
            elif 'rear_right' in img_name:
                pose_vc = T_vc_rear_right
            elif 'rear_left' in img_name:
                pose_vc = T_vc_rear_left
            else:
                print('error when reading pose_vc')
        T_vc = tf_matrix(pose_vc['rotation'], pose_vc['translation'])

        T_wc = T_wv @ T_vc
#         print(T_vc)
        t = T_wc[:3, 3]
#         print(t)
        
        line = img_name + ' ' + ' '.join([str(val) for val in t]) + '\n' 
        location_file.write(line)
#         print(line)
#         count += 1
#         if count == 3:
#              break
    

100%|██████████| 785/785 [00:00<00:00, 2565.23it/s]


```
colmap model_aligner --input_path ./1/ --output_path ./geo-registered-model/ --ref_images_path ~/work/netvlad_tf_open/python/db_location.txt --robust_alignment_max_error 0.5

Aligning reconstruction
-----------------------
 => Using 900 reference images
 => Alignment succeeded
 => Alignment error: 0.198891 (mean), 0.198105 (median)
 
Aligning reconstruction (after fix T)
-----------------------
 => Using 900 reference images
 => Alignment succeeded
 => Alignment error: 0.082774 (mean), 0.061503 (median)

```

## To look at the timestamps of Argoverse data
No continous sequences

In [24]:
import glob
from tqdm import tqdm
dataset_path = '/home/cloudlet/datasets/argoverse-tracking/train1/'
log_glob_list = sorted(glob.glob(dataset_path + '*'))
# print(log_glob_list)
log_list =[]
for log_dir in log_glob_list:
    log_id = log_dir.split('/')[-1]
    pose_json_dir = log_dir + '/poses/'
    pose_json_list = glob.glob(pose_json_dir + '*.json')
    pose_json_list = sorted(pose_json_list, key=lambda name: int(name[-23:-5]))
#     print(pose_json_list)
    first_ts = pose_json_list[0].split('/')[-1].split('_')[-1][:-5]
    last_ts = pose_json_list[-1].split('/')[-1].split('_')[-1][:-5]
    log = {'log_id': log_id, 'first': first_ts, 'last': last_ts}
    log_list.append(log)
#     print(log)
log_sorted = sorted(log_list, key=lambda x: int(x['first']))
print(log_sorted)
# with open(dataset_path + 'queries_with_intrinsics_1017_2.txt', 'w') as file:
#     image_glob_list = glob.glob(dataset_path + 'query/*.jpg')
#     for i, filename in enumerate(image_glob_list):
#         name = filename.split('/')[-1]

[{'log_id': '25952736-2595-2595-2595-225953853440', 'first': '315966391619461000', 'last': '315966406525337344'}, {'log_id': '6f153f9c-edc5-389f-ac6f-40705c30d97e', 'first': '315966434014592128', 'last': '315966449965293856'}, {'log_id': 'e17eed4f-3ffd-3532-ab89-41a3f24cf226', 'first': '315966968004148448', 'last': '315966983954851216'}, {'log_id': '043aeba7-14e5-3cde-8a5c-639389b6d3a6', 'first': '315967467001566824', 'last': '315967482952273448'}, {'log_id': '10b3a1d8-e56c-38be-aaf7-ef2f862a5c4e', 'first': '315967631819951000', 'last': '315967661910028992'}, {'log_id': '230970eb-dc2e-3133-b252-ff3c6f5d4284', 'first': '315967914019825000', 'last': '315967929944631856'}, {'log_id': '11953248-1195-1195-1195-511954366464', 'first': '315967953220225000', 'last': '315967983119616144'}, {'log_id': 'b3def699-884b-3c9e-87e1-1ab76c618e0b', 'first': '315967986019452848', 'last': '315968001970157032'}, {'log_id': '10b8dee6-778f-33e4-a946-d842d2d9c3d7', 'first': '315968229010579544', 'last': '3159